In [6]:
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
from cloudant.client import  Cloudant

from flask import Flask, request, render_template, redirect, url_for
import base64 


In [10]:
app=Flask(__name__)

client=Cloudant.iam('username','apikey',connect=True)
my_database = client['my_database']

image_folder=os.path.join('static','images')
app.config['UPLOAD_FOLDER'] = image_folder

@app.route('/')
def index():
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'drimage.jpg')
    return render_template('index.html',image=full_filename)

@app.route('/index')
def home():
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'drimage.jpg')
    return render_template('index.html',image=full_filename)

@app.route('/register')
def register():
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'registerimg.jpg')
    return render_template('register.html',image=full_filename)

@app.route('/afterreg',methods=['POST','GET'])
def afterreg():
    x=[x for x in request.form.values()]
    
    data={
        '_id':x[2],
        'name':x[0],
        'pwd':x[4],
        'email':x[1],
        'location':x[3]
    }
    query={'_id':{'$eq':data['_id']}}
    docs=my_database.get_query_result(query)
    
    if(len(docs.all())==0):
        url=my_database.create_document(data)
        full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'loginimg.jpg')
        return render_template('login.html',predict="Registration successfull please login using your credentials",image=full_filename)
    else:
        full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'registerimg.jpg')
        return render_template('register.html',pred="You are already a member login using your credentials",image=full_filename)

@app.route('/login')
def login():
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'loginimg.jpg')
    return render_template('login.html',image=full_filename)

@app.route('/afterlogin', methods=['POST','GET'])
def afterlogin():
    user=request.form['phoneno']
    passw=request.form['pwd']
    
    
    query={'_id':{'$eq':user}}
    docs=my_database.get_query_result(query)

    
 
    
    if(len(docs.all())==0):
        full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'loginimg.jpg')
        return render_template('login.html',predict="Phone number not found",image=full_filename)
    else:
        if((user==docs[0][0]['_id'] and passw==docs[0][0]['pwd'] )):
            full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'retina.jpg')
            full_filename1 = os.path.join(app.config['UPLOAD_FOLDER'], 'image6.png')
            return render_template('prediction.html',image=full_filename,image2=full_filename1)
        else:
            full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'loginimg.jpg')
            return render_template('login.html',image=full_filename,predict="Incorrect password")
            

    

@app.route('/prediction')
def prediction():
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'retina.jpg')
    full_filename1 = os.path.join(app.config['UPLOAD_FOLDER'], 'image6.png')
    return render_template('prediction.html',image=full_filename,image2=full_filename1)


@app.route('/logout')
def logout():
    return render_template('logout.html')

if __name__=="__main__":
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Oct/2022 19:05:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2022 19:05:02] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2022 19:05:14] "POST /afterreg HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2022 19:05:25] "POST /afterlogin HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2022 19:05:44] "GET /logout HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2022 19:05:47] "GET /index HTTP/1.1" 200 -
